In [1]:
#@title [Setup]
import ipywidgets as widgets
from IPython.display import Markdown as md
from IPython.display import display, Javascript
from IPython.core.getipython import get_ipython
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime
import requests
import duckdb


# Establish duckdb connection
db = duckdb.connect(database=':memory:', read_only=False)



# Praise Analysis and Visualization Dashboard
This document fetches the results of the praise reward system and allows analysis of the resulting reward distribution.

First, write the community you want to load praise from and click "Load Data". Some example communities are:
- https://praise.tecommons.org
- https://praise.giveth.io
- https://praise.commonsstack.org
- Or find more communities under https://explorer.givepraise.xyz/  !

Once the data is loaded, you can choose a period from the dropdown menu, or manually enter the dates yourself.

In [2]:
#@title _


# Function to load data from a list of tables into the duckdb database
def load_parquet(table_list):
    for table in table_list:
        url = table["path"]
        response = requests.get(url)

        parquet_file_path = "downloaded_file.parquet"

        with open(parquet_file_path, "wb") as f:
            f.write(response.content)

        df = pd.read_parquet(parquet_file_path)
        # Register the DataFrame as a DuckDB table
        db.register(table["name"], df)

# Callback function for the source_link combobox
def on_source_link_change(change):
    source_link = change['new']
    general_data = [
        {"name": "users_table", "path": f"{source_link}/api/users/export/parquet"},
        {"name": "period_table", "path": f"{source_link}/api/periods/export/parquet"},
        {"name": "useraccounts_table", "path": f"{source_link}/api/useraccounts/export/parquet"},
    ]
    load_parquet(general_data)
    update_period_dropdown()
    update_dates({'new': period_dropdown.value})

# Callback function for the period dropdown
def update_period_dropdown():
    global periods_df
    periods_df = db.execute("SELECT * FROM period_table").df()
    if periods_df.empty:
        period_dropdown.options = []
        start_date_picker.value = datetime(2023, 1, 1).date()
        end_date_picker.value = datetime(2023, 2, 1).date()
    else:
        period_dropdown.options = periods_df.name

# Callback function for the period dropdown
def update_dates(change):
    if periods_df.empty:
        start_date_picker.value = datetime(2023, 1, 1).date()
        end_date_picker.value = datetime(2023, 2, 1).date()
    else:
        selected_period = change.get('new', None) or period_dropdown.value
        row = periods_df.index[periods_df.name == selected_period]
        if row[0] == 0:
            start_date = datetime.strptime("2021-06-30T23:59:59.999Z", '%Y-%m-%dT%H:%M:%S.%fZ')
        else:
            start_date = pd.to_datetime(periods_df.iloc[row-1]['endDate'].values[0])
        end_date = pd.to_datetime(periods_df.iloc[row]['endDate'].values[0])
        start_date_picker.value = start_date.date()
        end_date_picker.value = end_date.date()


# Create widgets
source_link = widgets.Text(
    placeholder='Please write the URL',
    description='Community:',
    value= 'https://praise.tecommons.org',
    disabled=False
)

period_dropdown = widgets.Dropdown(description="Period")
start_date_picker = widgets.DatePicker(description="Start Date")
end_date_picker = widgets.DatePicker(description="End Date")

# Create button
load_button = widgets.Button(description="Load Data")

# Modify callback function to load data from the entered source link
def on_load_button_click(b):
    source_link_value = source_link.value.rstrip('/')  # Remove any trailing slashes
    general_data = [
        {"name": "users_table", "path": f"{source_link_value}/api/users/export/parquet"},
        {"name": "period_table", "path": f"{source_link_value}/api/periods/export/parquet"},
        {"name": "useraccounts_table", "path": f"{source_link_value}/api/useraccounts/export/parquet"},
    ]
    print(f"Loading data from: {source_link_value}")
    load_parquet(general_data)
    update_period_dropdown()
    update_dates({'new': period_dropdown.value})

# Bind the callback function to the button click event
load_button.on_click(on_load_button_click)
period_dropdown.observe(update_dates, names="value")

# Display widgets in a vertical container
hbox = widgets.HBox([source_link, load_button, period_dropdown, start_date_picker, end_date_picker])
display(hbox)

Loading data from: https://praise.tecommons.org


Now, please select the dates you want to load into the system. They will be stored in a dataframe called 'praise_df' for further analysis.

In [14]:
#@title (Sadly clicking this button won't work inside Google Colab. Instead, select the cell and press `CTRL + F10` )
def run_cells_below(btn):
    display(Javascript("Jupyter.notebook.execute_cells_below()"))

button = widgets.Button(description="Load Report")
button.on_click(run_cells_below)
display(button)

Button(description='Load Report', style=ButtonStyle())

In [15]:
#@title _

source_link_value = source_link.value.rstrip('/') # Remove any trailing slashes
start_date_value= datetime.strftime(start_date_picker.value, '%Y-%m-%dT%H:%M:%S.%fZ')
end_date_value = datetime.strftime(end_date_picker.value, '%Y-%m-%dT%H:%M:%S.%fZ')

selected_data =  [
    {"name": "praise_table", "path": f"{source_link_value}/api/praise/export/parquet?startDate={start_date_value[:10]}&endDate={end_date_value[:10]}"},
    {"name": "quantification_table", "path": f"{source_link_value}/api/quantifications/export/parquet?startDate={start_date_value[:10]}&endDate={end_date_value[:10]}"}
]

load_parquet(selected_data)

In [16]:
#@title General Information

# Set up helper praise_df DataFrame to be used in the following steps
praise_df = db.execute("""
SELECT pt.*, ua_giver.name AS giver_name, ua_receiver.name AS receiver_name
FROM praise_table  pt
JOIN useraccounts_table ua_giver ON pt.giver = ua_giver._id
JOIN useraccounts_table ua_receiver ON pt.receiver = ua_receiver._id
WHERE pt.createdAt BETWEEN ? AND ?
""", [start_date_picker.value, end_date_picker.value]).df()

## Clean usernames
praise_df['giver_name'] = praise_df['giver_name'].str.split('#').str[0]
praise_df['receiver_name'] = praise_df['receiver_name'].str.split('#').str[0]


md(f"<ul><li>This period covers praise given between <b>{start_date_value[:10]}</b> and  <b>{end_date_value[:10]}</b>. </li>  \
<li>There were <b>{len(praise_df.index)}</b> instances of praise given during this period. </li>  \
</ul>")


<ul><li>This period covers praise given between <b>2021-11-28</b> and  <b>2021-12-26</b>. </li>  <li>There were <b>1435</b> instances of praise given during this period. </li>  </ul>

### Rating distribution
Since praise gets valued on a scale, we can take a look at how often quantifiers award each value.
Note: This metric disregards scores of praise marked as a duplicate, since the score of the original is already being taken into account.

In [17]:
#@title _

hist = db.execute("""
    SELECT q.score, COUNT(*) as frequency
    FROM quantification_table q
    INNER JOIN praise_df p ON q.praise = p._id
    GROUP BY q.score
    ORDER BY q.score ASC
""").df()


fig_freq = go.Figure()

fig_freq.add_trace(go.Bar(
    x=hist['score'],
    y=hist['frequency'],
    name="Frequency",
    marker=dict(color='rgba(58, 71, 80, 0.6)')
))

fig_freq.update_layout(
    title="Praise Rating Distribution",
    width=800,
    height=300,
    xaxis=dict(
        title="Rating",
        type='category'
    ),
    yaxis=dict(
        title="Number of appearances"
    )
)


fig_freq.show()

### Top 10 highest rated contributions
The ten highest rated contributions for this round were the following:

In [18]:
#@title _
toppraise = db.execute("""
SELECT score AS Score, giver_name AS Giver, receiver_name AS Receiver, reason AS Reason
FROM praise_df p
ORDER BY p.score DESC
LIMIT 10;
""").df()
toppraise.style.hide(axis="index")

Score,Giver,Receiver,Reason
125.670000,Juankbell,"sem(🌸,🐝)",for developing and testing the smart contracts. They are actually hosting a demo of the augmented bonding curve and all the commons upgrade tooling. Much admiration and respect for that
68.000000,Mount Manu,mZ,for thought leadership in web3 and for keeping engineering ethics as our community’s North Star
63.500000,Juankbell,elessar.eth,for developing and testing the smart contracts. They are actually hosting a demo of the augmented bonding curve and all the commons upgrade tooling. Much admiration and respect for that
61.500000,Juankbell,gabi,for developing and testing the smart contracts. They are actually hosting a demo of the augmented bonding curve and all the commons upgrade tooling. Much admiration and respect for that
59.800000,iviangita,"sem(🌸,🐝)",for the incredible work they’re doing and making sure we have a safe launch for all of our smart contracts systems
59.330000,Juankbell,"sem(🌸,🐝)","for being one of the biggest makers of our DAO structure and tooling, and for being a big hearted person that cares for the success of the commons upgrade. Your participation is so important and it highlights specially for being silent"
57.130000,divine_comedian,"sem(🌸,🐝)","for med with the Commons Upgrade Demo making the \""meat\"" of the commons upgrade functional and secure"
55.000000,Juankbell,"sem(🌸,🐝)",for being a hero in the shadows and one of the biggest architects of the TEC
52.330000,Juankbell,bradleyc,for developing and testing the smart contracts. They are actually hosting a demo of the augmented bonding curve and all the commons upgrade tooling. Much admiration and respect for that
50.670000,Juankbell,Nuggan,for developing and testing the smart contracts. They are actually hosting a demo of the augmented bonding curve and all the commons upgrade tooling. Much admiration and respect for that


### Praise Giving Distribution
We can take a look at the amount of praise different users gave. The bar represents the amount of praise given by a specfic user, while the line shows the sum of the scores of that praise.


In [19]:
#@title _
giving_dist = db.execute("""
SELECT p.giver_name, COUNT(*) AS amount_given, AVG(p.score) AS avg_score
FROM praise_df p
GROUP BY p.giver_name
ORDER BY amount_given DESC
""").df()

fig_giving = go.Figure()

# Add the bar trace from fig_a
fig_giving.add_trace(go.Bar(
    x=giving_dist["giver_name"],
    y=giving_dist['amount_given'],
    name="Praise Given",
    marker_color='rgba(58, 71, 80, 0.6)',
    yaxis='y1'
))

# Add the line trace from fig_b
fig_giving.add_trace(go.Scatter(
    x=giving_dist["giver_name"],
    y=giving_dist['avg_score'],
    name="Average Score",
    mode='lines+markers',
    line=dict(color='red'),
    yaxis='y2'
))

# Configure the layout
fig_giving.update_layout(
    title="Praise Givers Sorted by Total Score and Average Score",
    xaxis_title="Giver",
    yaxis_title="Praise Given",
    yaxis2=dict(
        title="Average Score",
        overlaying='y',
        side='right'
    ),
    legend_title="",
    hovermode="x unified"
)

# Show the combined plot
fig_giving.show()

### Praise Receiver Distribution
We can also take a look at the amount of praise different users received. The bar represents the amount of praise instances received by a specfic user, while the point shows the average score of the received praise.

In [20]:
#@title _
receiving_dist = db.execute("""
SELECT p.receiver_name, COUNT(*) AS amount_received, AVG(p.score) AS avg_score
FROM praise_df p
GROUP BY p.receiver_name
ORDER BY amount_received DESC
""").df()
px.bar(receiving_dist, x="receiver_name",y='amount_received', labels={"receiver_name": "Receiver","amount_received": "Praise Received"},title='Praise Receivers Sorted by Total Score')

# Assuming 'giving_dist' is a pandas DataFrame with columns: 'giver_name', 'amount_given', 'avg_score'
# You might need to modify the code below to match your data format.

# Create a combined Figure object
fig_receiving = go.Figure()

# Add the bar trace from fig_a
fig_receiving.add_trace(go.Bar(
    x=receiving_dist["receiver_name"],
    y=receiving_dist['amount_received'],
    name="Praise Received",
    marker_color='rgba(58, 71, 80, 0.6)',
    yaxis='y1'
))

# Add the line trace from fig_b
fig_receiving.add_trace(go.Scatter(
    x=receiving_dist["receiver_name"],
    y=receiving_dist['avg_score'],
    name="Average Score",
    mode='lines+markers',
    line=dict(color='red'),
    yaxis='y2'
))

# Configure the layout
fig_receiving.update_layout(
    title="Praise Givers Sorted by Total Score and Average Score",
    xaxis_title="Receiver",
    yaxis_title="Praise Received",
    yaxis2=dict(
        title="Average Score",
        overlaying='y',
        side='right'
    ),
    legend_title="",
    hovermode="x unified"
)

# Show the combined plot
fig_receiving.show()

## Amount of praise over time
Here we can observe the amount of praise dished on a weekly basis

In [21]:
#@title _
praise_over_time = db.execute("""
SELECT DATE_TRUNC('week', createdAt) AS week_start,
       COUNT(*) AS row_count
FROM praise_df
GROUP BY week_start
ORDER BY week_start;
""").df()
fig_praise_over_time = px.line(praise_over_time, x="week_start",y='row_count', labels={"week_start": "Weeks","row_count": "Amount of Praise"},  title='Praise Given Over Time')
# Update the line color and fill below the line
fig_praise_over_time.update_traces(line=dict(color='rgba(58, 71, 80, 0.6)'))
fig_praise_over_time.update_traces(fill='tozeroy', fillcolor='rgba(58, 71, 80, 0.6)')

fig_praise_over_time.show()

## Givers and receivers over time
Here we can observe the amount of unique
users who either dished or received praise on any given week.

In [22]:
#@title _
participants_over_time = db.execute("""
SELECT DATE_TRUNC('week', createdAt) AS week_start_date,
       COUNT(DISTINCT giver) AS unique_givers,
       COUNT(DISTINCT receiver) AS unique_receivers
FROM praise_df
GROUP BY week_start_date
ORDER BY week_start_date;
""").df()
fig_participants_over_time = px.line(participants_over_time, x="week_start_date",y=['unique_givers','unique_receivers'], labels={"week_start_date": "Weeks","value": "Amount of Participants", "unique_givers":"Unique Givers", "unique_receivers":"Unique Receivers"}, title='Participants Over Time')
fig_participants_over_time.update_traces(fill='tozeroy', fillcolor='rgba(58, 71, 80, 0.3)')
fig_participants_over_time.show()

### Praise Flow

Now for something more fun: let's surface the top "praise flows" from the data.
On one side we have the top 15 praise givers separately, on the other the top 20 receivers. The people outside the selection get aggregated into the "REST FROM" and "REST TO" categories.

In [23]:
#@title _

# STEP 1: PREPARE DATA

def prepare_praise_flow(dataframe_in, n_senders, n_receivers):
    # Select required columns and drop rows with missing values
    reference_df = dataframe_in[['FROM', 'TO', 'TOTAL_SCORE']].dropna().reset_index(drop=True)

    # Select top n_senders senders by combined TOTAL_SCORE and rename the rest as 'Rest from 1'
    sender_scores = reference_df.groupby('FROM')['TOTAL_SCORE'].sum()
    top_senders = sender_scores.nlargest(n_senders).index
    reference_df.loc[~reference_df['FROM'].isin(top_senders), 'FROM'] = 'REST FROM'

    # Select top n_receivers receivers by combined TOTAL_SCORE and rename the rest as 'Rest to 1'
    receiver_scores = reference_df.groupby('TO')['TOTAL_SCORE'].sum()
    top_receivers = receiver_scores.nlargest(n_receivers).index
    reference_df.loc[~reference_df['TO'].isin(top_receivers), 'TO'] = 'REST TO'

    # Group by sender and receiver, and sum up the scores
    df4 = reference_df.groupby(['FROM', 'TO']).sum().reset_index()

    # Append a space to the 'TO' column
    df4['TO'] = df4['TO'] + ' '

    return df4



NUMBER_OF_SENDERS_FLOW = 15 #The left side, the praise senders. X largest ones + one bucket for the rest
NUMBER_OF_RECEIVERS_FLOW = 20 #The right side, the praise receivers. X largest ones + one bucket for the rest
#dist_for_praise_flow = praise_distribution.rename(columns = {'FROM USER ACCOUNT':'FROM', 'TO USER ACCOUNT':'TO'})

praise_flow = db.execute("""
SELECT giver_name as 'FROM' , receiver_name as 'TO', SUM(score) as 'TOTAL_SCORE'
FROM praise_df
GROUP BY giver_name, receiver_name
""").df()

praise_flow = prepare_praise_flow(praise_flow.copy(), n_senders=NUMBER_OF_SENDERS_FLOW, n_receivers=NUMBER_OF_RECEIVERS_FLOW)


# STEP 2: CREATE SANKEY GRAPH

# Get unique labels for nodes (both 'FROM' and 'TO')
labels = list(set(praise_flow['FROM']).union(set(praise_flow['TO'])))

# Get index of labels
source_indices = [labels.index(src) for src in praise_flow['FROM']]
target_indices = [labels.index(trg) for trg in praise_flow['TO']]

# Create a color mapping for the 'FROM' values
from_labels = praise_flow['FROM'].unique()
colors = px.colors.qualitative.Safe
color_mapping = {label: colors[i % len(colors)] for i, label in enumerate(from_labels)}

# Assign colors to nodes based on 'FROM' values
node_colors = ['rgba(100, 100, 100, 0.5)'] * len(labels)
for idx, label in enumerate(labels):
    if label in color_mapping:
        node_colors[idx] = color_mapping[label]

# Create Sankey diagram
fig_praise_flow = go.Figure(data=[go.Sankey(
    node=dict(
        pad=0,
        thickness=40,
        line=dict(color="black", width=0.5),
        label=labels,
        color=node_colors
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=praise_flow['TOTAL_SCORE'],
        color=[color_mapping[src] for src in praise_flow['FROM']]
    )
)])

# Configure layout
fig_praise_flow.update_layout(
    title_text="Praise flow for Batch 1. Sum of Praise. Left - praise sender. Right - praise receiver",
    font_size=10,
    width=1000,
    height=800,
    plot_bgcolor='snow',
    margin=dict(l=30, r=30, t=50, b=30)
)

# Show the plot
fig_praise_flow.show()

## New Givers
Let's see the amount of people who praise for the first time on a weekly basis.

In [24]:
#@title _
new_givers = db.execute("""
SELECT DATE_TRUNC('week', sub.createdAt) as week_start,
       COUNT(DISTINCT sub.giver) as num_new_givers
FROM (SELECT giver, MIN(createdAt) as createdAt
      FROM praise_df
      GROUP BY giver) as sub
WHERE sub.createdAt >= ?
GROUP BY week_start
ORDER BY week_start ASC
""", [start_date_value]).df()

fig_new_givers = px.line(new_givers, x="week_start",y='num_new_givers', labels={"week_start": "Weeks","num_new_givers": "New Givers"}, title='New Givers Over Time')
# Update the line color and fill below the line
fig_new_givers.update_traces(line=dict(color='rgba(58, 71, 80, 0.6)'))
fig_new_givers.update_traces(fill='tozeroy', fillcolor='rgba(58, 71, 80, 0.6)')
fig_new_givers.show()
